In [1]:
import os
import re
import sys
import torch
import hashlib
import itertools
import logging
import numpy as np

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


from progress.bar import Bar
from concurrent.futures import ProcessPoolExecutor

from Utils.NotesSeq import NoteSeq as ns
from Utils.EventSeq import EventSeq as es
from Utils.ControlSeq import ControlSeq as cs
from Utils import utils

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical, Gumbel
from torch import optim

from config import device

In [3]:
from pretty_midi import PrettyMIDI, Note, Instrument

# On details of data

In [4]:
def preprocess_midi(path):

    note_seq = ns.from_midi_file(path)
    iter = itertools.cycle(note_seq)

    es1 = list
    es2 = list

    cs1 = list
    cs2 = list
    ev = True
    #print("evaluating")
    for seq in iter:
        #print(seq)
        if ev:
            #print("1st iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es1 = event_seq.to_array()
            cs1 = control_seq.to_compressed_array()
            ev = False
        else:
            #print("2nd iter")
            seq.adjust_time(-seq.notes[0].start)
            event_seq = es.from_note_seq(seq)
            control_seq = cs.from_event_seq(event_seq)
            es2 = event_seq.to_array()
            cs2 = control_seq.to_compressed_array()
            break

    return es1, es2, cs1, cs2

In [5]:
midi_path = './Naruto_Shippuden_-_Silhouette.mid'

In [6]:
def from_tracks(midi, flag, programs=range(128)):
    group_indices = flag # Variable flag permits to manage different tracks
    group_notes = itertools.chain(*[
        midi.instruments[i].notes for i in group_indices
    ])
    #print(list(group_notes))
    #print(list(group_notes))

    temp = list(group_notes)
    #print(temp)
    #print(70*'-')
    for note in temp:
        assert isinstance(note, Note)
    notes = list(filter(lambda note: note.end >= note.start, temp)) # filtro per essere sicuri che la nota estratta sia qurlla corretta
    #notes = notes.sort(key=lambda note: note.start)
    #print(notes)

    return ns(list(notes))

In [7]:
def from_midi_file(path):
        midi = PrettyMIDI(path)
        track_len = len(midi.instruments)
        print(midi.instruments)
        if(track_len < 3):
            stream1 = from_tracks(midi,[0])
            print(stream1.notes)
            stream2 = from_tracks(midi,[1])
            #print(" 2 tracce : " + path)
        elif(track_len < 4):
            stream1 = from_tracks(midi,[0])
            stream2 = from_tracks(midi,[1,2])
            #print(" 3 tracce : " + path)
        else:
            stream1 = from_tracks(midi,[0,1])
            stream2 = from_tracks(midi,[2,3])
            #print(" 4 tracce : " + path)

        return stream1,stream2

In [8]:
midi = PrettyMIDI(midi_path)
track_len = len(midi.instruments)

In [9]:
midi.instruments

[Instrument(program=0, is_drum=False, name=""),
 Instrument(program=0, is_drum=False, name="")]

In [10]:
s1,s2 = from_midi_file(midi_path)

[Instrument(program=0, is_drum=False, name=""), Instrument(program=0, is_drum=False, name="")]
[Note(start=0.000000, end=0.153379, pitch=59, velocity=112), Note(start=0.162163, end=0.315541, pitch=57, velocity=112), Note(start=0.324325, end=0.477704, pitch=59, velocity=112), Note(start=0.486488, end=0.793921, pitch=62, velocity=112), Note(start=0.810813, end=0.964191, pitch=57, velocity=112), Note(start=0.972975, end=1.126354, pitch=59, velocity=112), Note(start=1.135138, end=1.288516, pitch=57, velocity=112), Note(start=1.297300, end=1.450679, pitch=59, velocity=112), Note(start=1.459463, end=1.766896, pitch=62, velocity=112), Note(start=1.783788, end=1.937166, pitch=57, velocity=112), Note(start=1.945950, end=2.253383, pitch=59, velocity=112), Note(start=2.594600, end=3.056087, pitch=69, velocity=112), Note(start=2.594600, end=3.056087, pitch=74, velocity=112), Note(start=3.081088, end=3.542575, pitch=69, velocity=112), Note(start=3.567575, end=3.875008, pitch=74, velocity=112), Note

In [11]:
print(s2.notes)

[Note(start=0.000000, end=0.153379, pitch=35, velocity=96), Note(start=0.000000, end=0.153379, pitch=47, velocity=96), Note(start=0.162163, end=0.315541, pitch=35, velocity=96), Note(start=0.162163, end=0.315541, pitch=47, velocity=96), Note(start=0.324325, end=0.477704, pitch=35, velocity=96), Note(start=0.324325, end=0.477704, pitch=47, velocity=96), Note(start=0.486488, end=0.639866, pitch=33, velocity=96), Note(start=0.486488, end=0.639866, pitch=45, velocity=96), Note(start=0.648650, end=0.802029, pitch=33, velocity=96), Note(start=0.648650, end=0.802029, pitch=45, velocity=96), Note(start=0.810813, end=0.964191, pitch=33, velocity=96), Note(start=0.810813, end=0.964191, pitch=45, velocity=96), Note(start=0.972975, end=1.126354, pitch=33, velocity=96), Note(start=0.972975, end=1.126354, pitch=45, velocity=96), Note(start=1.135138, end=1.288516, pitch=33, velocity=96), Note(start=1.135138, end=1.288516, pitch=45, velocity=96), Note(start=1.297300, end=1.450679, pitch=31, velocity=9

In [12]:
s1.adjust_time(-s1.notes[0].start)

In [13]:
USE_VELOCITY = True
DEFAULT_TEMPO = 120
BEAT_LENGTH = 60 / DEFAULT_TEMPO
DEFAULT_TIME_SHIFT_BINS = 1.15 ** np.arange(32) / 65
DEFAULT_VELOCITY_STEPS = 32
DEFAULT_NOTE_LENGTH = BEAT_LENGTH * 2
MIN_NOTE_LENGTH = BEAT_LENGTH / 2
DEFAULT_PITCH_RANGE = range(21, 109)
DEFAULT_VELOCITY_RANGE = range(21, 109)

In [14]:
pitch_range = DEFAULT_PITCH_RANGE
velocity_range = DEFAULT_VELOCITY_RANGE
velocity_steps = DEFAULT_VELOCITY_STEPS
time_shift_bins = DEFAULT_TIME_SHIFT_BINS

In [15]:
BEAT_LENGTH

0.5

In [16]:
time_shift_bins

array([0.01538462, 0.01769231, 0.02034615, 0.02339808, 0.02690779,
       0.03094396, 0.03558555, 0.04092338, 0.04706189, 0.05412117,
       0.06223935, 0.07157525, 0.08231154, 0.09465827, 0.10885701,
       0.12518556, 0.1439634 , 0.16555791, 0.19039159, 0.21895033,
       0.25179288, 0.28956182, 0.33299609, 0.3829455 , 0.44038733,
       0.50644542, 0.58241224, 0.66977407, 0.77024019, 0.88577621,
       1.01864265, 1.17143904])

In [17]:
MIN_NOTE_LENGTH

0.25

In [18]:
def get_velocity_bins():
        n = es.velocity_range.stop - es.velocity_range.start
        return np.arange(es.velocity_range.start,
                         es.velocity_range.stop,
                         n / (es.velocity_steps - 1))

In [19]:
get_velocity_bins()

array([ 21.        ,  23.83870968,  26.67741935,  29.51612903,
        32.35483871,  35.19354839,  38.03225806,  40.87096774,
        43.70967742,  46.5483871 ,  49.38709677,  52.22580645,
        55.06451613,  57.90322581,  60.74193548,  63.58064516,
        66.41935484,  69.25806452,  72.09677419,  74.93548387,
        77.77419355,  80.61290323,  83.4516129 ,  86.29032258,
        89.12903226,  91.96774194,  94.80645161,  97.64516129,
       100.48387097, 103.32258065, 106.16129032])

In [20]:
class Event:

    def __init__(self, type, time, value):
        self.type = type # can be of four types: 1)velocity 2)note_on 3)note_off 4)time_shift
        self.time = time # indicates the absoulte time where the event starts
        self.value = value # in case of velocity it indicates the velocity bin, in case of pitch the actual pitch

    def __repr__(self):
        return 'Event(type={}, time={}, value={})'.format(
            self.type, self.time, self.value)

In [21]:
def from_note_seq(note_seq):
    note_events = []
    
    velocity_bins = get_velocity_bins()

    for note in note_seq.notes:
        if note.pitch in pitch_range:

            # Regularization of the velocity of a note. We impose the maximum or minimum velocity in 
            # case the original one is out of our bounds
            velocity = note.velocity
            velocity = max(velocity, velocity_range.start)
            velocity = min(velocity, velocity_range.stop - 1)

            velocity_index = np.searchsorted(velocity_bins, velocity)
            note_events.append(Event('velocity', note.start, velocity_index))

            pitch_index = note.pitch - pitch_range.start
            note_events.append(Event('note_on', note.start, pitch_index))
            note_events.append(Event('note_off', note.end, pitch_index))

    note_events.sort(key=lambda event: event.time)  # stable

    #print(note_events)

    events = []

    for i, event in enumerate(note_events):
        events.append(event)
        #print(event)

        if event is note_events[-1]:
            break

        interval = note_events[i + 1].time - event.time
        shift = 0

        while interval - shift >= time_shift_bins[0]:
            index = np.searchsorted(time_shift_bins,
                                    interval - shift, side='right') - 1
            events.append(Event('time_shift', event.time + shift, index))
            shift += time_shift_bins[index]

    return events

In [22]:
pure_events = from_note_seq(s1)

In [23]:
for event in pure_events:
    assert isinstance(event, Event)

In [24]:
print(type(pure_events[0]))

<class '__main__.Event'>


In [25]:
es_1 = es(pure_events)

In [26]:
class Control:

    def __init__(self, pitch_histogram, note_density):
        self.pitch_histogram = pitch_histogram  # list
        self.note_density = note_density  # int

    def __repr__(self):
        return 'Control(pitch_histogram={}, note_density={})'.format(
            self.pitch_histogram, self.note_density)

    def to_array(self):
        feat_dims = cs.feat_dims()
        ndens = np.zeros([feat_dims['note_density']])
        ndens[self.note_density] = 1.  # [dens_dim]
        phist = np.array(self.pitch_histogram)  # [hist_dim]
        return np.concatenate([ndens, phist], 0)  # [dens_dim + hist_dim]


In [27]:
start, end = 0, 0
pitch_count = np.zeros([12])
note_count = 0

controls = []
def _rel_pitch(pitch):
            return (pitch - 24) % 12

events = list(es_1.events)

In [28]:
for i, event in enumerate(events):
    #print(i)
    #print(event)
    #print(f"Start at {start} and Ending at {end}. Cycle {i}/{len(events)}. Note Count {note_count}")

    while start < i:
        if events[start].type == 'note_on':
            abs_pitch = events[start].value + es.pitch_range.start
            rel_pitch = _rel_pitch(abs_pitch)
            pitch_count[rel_pitch] -= 1.
            note_count -= 1.
        start += 1
        #print(f"Start Value = {start} Abs pitch = {abs_pitch} Rel pitch = {rel_pitch}")

    while end < len(events):
        if events[end].time - event.time > cs.window_size:
            break
        if events[end].type == 'note_on':
            abs_pitch = events[end].value + es.pitch_range.start
            rel_pitch = _rel_pitch(abs_pitch)
            pitch_count[rel_pitch] += 1.
            note_count += 1.
        end += 1
        #print(f"End Value = {end} Abs pitch = {abs_pitch} Rel pitch = {rel_pitch}")
    # list containing the probability of each note (sum = 1)
    pitch_histogram = (
                pitch_count / note_count
                if note_count
                else np.ones([12]) / 12
            ).tolist()
    #print(pitch_histogram)

    note_density = max(np.searchsorted(
        cs.note_density_bins,
        note_count, side='right') - 1, 0)
    #print(note_density)

    controls.append(Control(pitch_histogram, note_density))

In [29]:
cs_1 = cs.from_event_seq(es_1)

In [30]:
esa = es_1.to_array()

In [31]:
feat_idxs = es.feat_ranges()

In [32]:
feat_idxs['note_on']

range(0, 88)

In [33]:
idxs = [feat_idxs[event.type][event.value] for event in events]

In [34]:
# taken from one file
#
# 88 ids for note on (88 pitches)
# 88 ids for note off (88 pitches)
# 32 ids for velocity
# 32 ids for time shifts

#for event in events:
#    print(f"event of type {event.type} in range {feat_idxs[event.type]}")#[event.value])

In [35]:
dtype = np.uint8 if es.dim() <= 256 else np.uint16

In [36]:
dtype

numpy.uint8

In [37]:
es1 = np.array(idxs, dtype=dtype)

In [38]:
ndens = [control.note_density for control in cs_1.controls]
ndens = np.array(ndens, dtype=np.uint8).reshape(-1, 1)

#densita di note per ogni step

ndens

array([[3],
       [3],
       [3],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [39]:
phist = [control.pitch_histogram for control in cs_1.controls]
phist = (np.array(phist) * 255).astype(np.uint8)

phist[0]

array([  0,   0,  46,   0,   0,   0,   0,   0,   0,  92,   0, 115],
      dtype=uint8)

In [40]:
cs_fin = np.concatenate([
            ndens,  # [steps, 1] density index
            phist  # [steps, hist_dim] 0-255
        ], 1)  # [steps, hist_dim + 1]

cs_fin[0] #

array([  3,   0,   0,  46,   0,   0,   0,   0,   0,   0,  92,   0, 115],
      dtype=uint8)

# Quick multi stream

# Taking streams

In [41]:
midi_path = './Naruto_Shippuden_-_Silhouette.mid'

In [42]:
midi = PrettyMIDI(midi_path)

In [43]:
es1, es2, cs1, cs2 = preprocess_midi(midi_path)

## settings for training and model

In [44]:
# Variables for training process

init_dim = 32
event_dim = es.dim()
control_dim = cs.dim()
hidden_dim = 512
gru_layers = 3
gru_droput = 0.3

In [45]:
learning_rate = 0.001
batch_size = 64
window_size = 200
stride_size = 10
use_transposition = False
control_ratio = 1.0
teacher_forcing_ratio = 1.0

## Options for training session

In [46]:
sess_path = 'save/train.sess'
data_path = 'Processed-RAW'
saving_interval = 60.
reset_optimizer = False
enable_logging = False

# Dataset

In [47]:
class Dataset:
    def __init__(self, root, verbose=False):
        assert os.path.isdir(root), root
        paths = utils.find_files_by_extensions(root, ['.data'])

        self.root = root
        self.samples = []
        self.seqlens = []
        self.samples2 = []
        self.seqlens2 = []

        if verbose:
            paths = Bar(root).iter(list(paths))
        for path in paths:
            eventseq, eventseq2, controlseq, controlseq2 = torch.load(path)
            controlseq = cs.recover_compressed_array(controlseq)
            controlseq2 = cs.recover_compressed_array(controlseq2)
            assert len(eventseq) == len(controlseq)
            assert len(eventseq2) == len(controlseq2)
            self.samples.append((eventseq, controlseq))
            self.seqlens.append(len(eventseq))
            self.samples2.append((eventseq2,controlseq2))
            self.seqlens2.append(len(eventseq2))

        self.avglen = np.mean(self.seqlens)
        self.avglen2 = np.mean(self.seqlens2)
    
    def batches(self, batch_size, window_size, stride_size):
        indeces = [(i, range(j, j + window_size))
                   for i, seqlen in enumerate(self.seqlens)
                   for j in range(0, seqlen - window_size, stride_size)]
        while True:
            eventseq_batch = []
            controlseq_batch = []
            eventseq_batch2 = []
            controlseq_batch2 = []
            n = 0
            for ii in np.random.permutation(len(indeces)):
                i, r = indeces[ii]

                eventseq, controlseq = self.samples[i]
                eventseq2, controlseq2 = self.samples2[i]

                eventseq = eventseq[r.start:r.stop]
                eventseq2 = eventseq2[r.start:r.stop]

                controlseq = controlseq[r.start:r.stop]
                controlseq2 = controlseq2[r.start:r.stop]

                eventseq_batch.append(eventseq)
                controlseq_batch.append(controlseq)
                eventseq_batch2.append(eventseq2)
                controlseq_batch2.append(controlseq2)

                n += 1
                if n == batch_size:
                    yield (np.stack(eventseq_batch, axis=1),
                           np.stack(controlseq_batch, axis=1),
                           np.stack(eventseq_batch, axis=1),
                           np.stack(controlseq_batch, axis=1))
                    eventseq_batch.clear()
                    controlseq_batch.clear()
                    eventseq_batch2.clear()
                    controlseq_batch2.clear()
                    n = 0
    
    def __repr__(self):
        return (f'Dataset(root="{self.root}", '
                f'samples={len(self.samples)}, '
                f'avglen={self.avglen})')


In [48]:
cs1 = cs.recover_compressed_array(cs1)
cs2 = cs.recover_compressed_array(cs2)

In [49]:
seqlens = []
seqlens.append(len(es1))

In [50]:
indeces = [(i, range(j, j + window_size))
                for i, seqlen in enumerate(seqlens)
                for j in range(0, seqlen - window_size, stride_size)]

In [51]:
indeces[0]

(0, range(0, 200))

In [52]:
indeces[1]

(0, range(10, 210))

In [53]:
for ii in np.random.permutation(len(indeces)):
    i, r = indeces[ii]
    print(r)
    print(r.start)
    print(r.stop)
    break

range(860, 1060)
860
1060


In [54]:
dataset = Dataset(data_path, verbose=True)

In [55]:
dataset_size = len(dataset.samples)
assert dataset_size > 0

# Models Performance 2X

In [56]:
class P2X(nn.Module):

    def _initialize_weights(self):
        nn.init.xavier_normal_(self.event_embedding.weight)
        nn.init.xavier_normal_(self.inithid_fc.weight)
        self.inithid_fc.bias.data.fill_(0.)
        nn.init.xavier_normal_(self.concat_input_fc.weight)
        nn.init.xavier_normal_(self.output_fc.weight)
        self.output_fc.bias.data.fill_(0.)

    def __init__(self, event_dim, control_dim, init_dim, hidden_dim,
                 inithid_fc = None, gru_layers=3, gru_dropout=0.3):
        super().__init__()

        # Parameters initialization
        self.event_dim = event_dim
        self.control_dim = control_dim
        self.init_dim = init_dim
        self.hidden_dim = hidden_dim
        self.gru_layers = gru_layers
        self.concat_dim = event_dim + 1 + control_dim
        self.input_dim = hidden_dim
        self.output_dim = event_dim

        self.primary_event = self.event_dim - 1

        #Model Layers

        self.inithid_fc = nn.Linear(init_dim, gru_layers * hidden_dim)
        self.inithid_fc_activation = nn.Tanh()

        self.event_embedding = nn.Embedding(event_dim, event_dim)
        self.concat_input_fc = nn.Linear(self.concat_dim, self.input_dim)
        self.concat_input_fc_activation = nn.LeakyReLU(0.1, inplace=True)

        self.gru = nn.GRU(self.input_dim, self.hidden_dim,
                          num_layers=gru_layers, dropout=gru_dropout)
        self.output_fc = nn.Linear(hidden_dim * gru_layers, self.output_dim)
        self.output_fc_activation = nn.Softmax(dim=-1)

        self._initialize_weights()

    def forward(self, event, control=None, hidden=None):
        # One step forward

        assert len(event.shape) == 2
        assert event.shape[0] == 1
        batch_size = event.shape[1]
        event = self.event_embedding(event)

        if control is None:
            default = torch.ones(1, batch_size, 1).to(device)
            control = torch.zeros(1, batch_size, self.control_dim).to(device)
        else:
            default = torch.zeros(1, batch_size, 1).to(device)
            assert control.shape == (1, batch_size, self.control_dim)

        concat = torch.cat([event, default, control], -1)
        input = self.concat_input_fc(concat)  #nn.Linear(self.concat_dim, self.input_dim)
        input = self.concat_input_fc_activation(input)  #nn.LeakyReLU(0.1, inplace=True)

        _, hidden = self.gru(input, hidden)  #nn.GRU(self.input_dim, self.hidden_dim,num_layers=gru_layers, dropout=gru_dropout)

        output = hidden.permute(1, 0, 2).contiguous()
        output = output.view(batch_size, -1).unsqueeze(0)
        output = self.output_fc(output) #nn.Linear(hidden_dim * gru_layers, self.output_dim)
        return output, hidden # output is under the form of a logit
    
    def _sample_event(self, output, greedy=True, temperature=1.0):
        if greedy:
            return output.argmax(-1)
        else:
            output = output / temperature
            probs = self.output_fc_activation(output)
            return Categorical(probs).sample()
    
    def get_primary_event(self, batch_size):
        return torch.LongTensor([[self.primary_event] * batch_size]).to(device)

    def init_to_hidden(self, init):
        # [batch_size, init_dim]
        batch_size = init.shape[0]
        out = self.inithid_fc(init)
        out = self.inithid_fc_activation(out)
        out = out.view(self.gru_layers, batch_size, self.hidden_dim)
        return out
    
    def expand_controls(self, controls, steps):
        # [1 or steps, batch_size, control_dim]
        assert len(controls.shape) == 3
        assert controls.shape[2] == self.control_dim
        if controls.shape[0] > 1:
            assert controls.shape[0] >= steps
            return controls[:steps]
        return controls.repeat(steps, 1, 1)
    
    def generate(self, init, batch_size, init_dim, steps, events = None, controls = None,
                 verbose = True, greedy = 1.0, temperature = 1.0):
        batch_size = batch_size
        self.init_dim = init_dim

        assert init.shape[1] == self.init_dim
        assert steps > 0

        use_teacher_forcing = events is not None

        if use_teacher_forcing:
            assert len(events.shape) == 2
            assert events.shape[0] >= steps - 1
            events = events[:steps-1]

        event = self.get_primary_event(batch_size)

        use_control = controls is not None

        if use_control:
            controls = self.expand_controls(controls, steps)
        hidden = self.init_to_hidden(init)

        outputs = []
        step_iter = range(steps)

        if verbose:
            step_iter = Bar('Generating').iter(step_iter)

        for step in step_iter:
            #control = controls[step].unsqueeze(0) if use_control else None
            control = None
            output, hidden = self.forward(event, control, hidden)

            use_greedy = np.random.random() < greedy
            event = self._sample_event(output, greedy=use_greedy,
                                       temperature=temperature)
            
            #here outputs are served in the lo-git format
            outputs.append(output)
            #
            #if use_teacher_forcing and step < steps - 1: # avoid last one
            #    if np.random.random() <= teacher_forcing_ratio:
            #        event = events[step].unsqueeze(0)
        
        return torch.cat(outputs, 0)
        

class P2XSecondary(nn.Module):

    def _initialize_weights(self):
        nn.init.xavier_normal_(self.event_embedding.weight)
        nn.init.xavier_normal_(self.inithid_fc.weight)
        self.inithid_fc.bias.data.fill_(0.)
        nn.init.xavier_normal_(self.concat_input_fc.weight)
        nn.init.xavier_normal_(self.output_fc.weight)
        self.output_fc.bias.data.fill_(0.)

    def __init__(self, event_dim, control_dim, init_dim, hidden_dim,
                 inithid_fc = None, gru_layers=3, gru_dropout=0.3):
        super().__init__()

        # Parameters initialization
        self.event_dim = event_dim
        self.control_dim = control_dim
        self.init_dim = init_dim
        self.hidden_dim = hidden_dim
        self.gru_layers = gru_layers
        self.concat_dim = event_dim + 1 + control_dim
        self.input_dim = hidden_dim
        self.output_dim = event_dim

        self.primary_event = self.event_dim - 1

        #Model Layers

        self.inithid_fc = nn.Linear(init_dim, gru_layers * hidden_dim)
        self.inithid_fc_activation = nn.Tanh()

        self.event_embedding = nn.Embedding(event_dim, event_dim)
        self.concat_input_fc = nn.Linear(self.concat_dim, self.input_dim)
        self.concat_input_fc_activation = nn.LeakyReLU(0.1, inplace=True)

        self.gru = nn.GRU(self.input_dim, self.hidden_dim,
                          num_layers=gru_layers, dropout=gru_dropout)
        self.output_fc = nn.Linear(hidden_dim * gru_layers, self.output_dim)
        self.output_fc_activation = nn.Softmax(dim=-1)

        self._initialize_weights()

    def forward(self, event, outputA, control=None, hidden=None):
        # One step forward

        assert len(event.shape) == 2
        assert event.shape[0] == 1
        batch_size = event.shape[1]
        event = self.event_embedding(event)

        if control is None:
            default = torch.ones(1, batch_size, 1).to(device)
            control = torch.zeros(1, batch_size, self.control_dim).to(device)
        else:
            default = torch.zeros(1, batch_size, 1).to(device)
            assert control.shape == (1, batch_size, self.control_dim)

        concat = torch.cat([event, default, control], -1)
        input = self.concat_input_fc(concat)  #nn.Linear(self.concat_dim, self.input_dim)
        input = self.concat_input_fc_activation(input)  #nn.LeakyReLU(0.1, inplace=True)

        _, hidden = self.gru(input, hidden)  #nn.GRU(self.input_dim, self.hidden_dim,num_layers=gru_layers, dropout=gru_dropout)

        output = hidden.permute(1, 0, 2).contiguous()
        output = output.view(batch_size, -1).unsqueeze(0)
        output = self.output_fc(output) #nn.Linear(hidden_dim * gru_layers, self.output_dim)

        return output, hidden # output is under the form of a logit
    
    def _sample_event(self, output, greedy=True, temperature=1.0):
        if greedy:
            return output.argmax(-1)
        else:
            output = output / temperature
            probs = self.output_fc_activation(output)
            return Categorical(probs).sample()
    
    def get_primary_event(self, batch_size):
        return torch.LongTensor([[self.primary_event] * batch_size]).to(device)

    def init_to_hidden(self, init):
        # [batch_size, init_dim]
        batch_size = init.shape[0]
        out = self.inithid_fc(init)
        out = self.inithid_fc_activation(out)
        out = out.view(self.gru_layers, batch_size, self.hidden_dim)
        return out
    
    def expand_controls(self, controls, steps):
        # [1 or steps, batch_size, control_dim]
        assert len(controls.shape) == 3
        assert controls.shape[2] == self.control_dim
        if controls.shape[0] > 1:
            assert controls.shape[0] >= steps
            return controls[:steps]
        return controls.repeat(steps, 1, 1)
    
    def generate(self, init, batch_size, init_dim, steps, events = None, controls = None, outputA = None,
                 verbose = True, greedy = 1.0, temperature = 1.0):
        batch_size = batch_size
        self.init_dim = init_dim

        assert init.shape[1] == self.init_dim
        assert steps > 0

        use_teacher_forcing = events is not None

        if use_teacher_forcing:
            assert len(events.shape) == 2
            assert events.shape[0] >= steps - 1
            events = events[:steps-1]

        event = self.get_primary_event(batch_size)

        use_control = controls is not None

        if use_control:
            controls = self.expand_controls(controls, steps)
        hidden = self.init_to_hidden(init)

        outputs = []
        step_iter = range(steps)

        if verbose:
            step_iter = Bar('Generating').iter(step_iter)

        for step in step_iter:
            #control = controls[step].unsqueeze(0) if use_control else None
            control = None
            output, _ = self.forward(event,outputA, control, hidden)

            use_greedy = np.random.random() < greedy
            event = self._sample_event(output, greedy=use_greedy,
                                       temperature=temperature)
            
            #here outputs are served in the logit format
            outputs.append(output)
            #
            #if use_teacher_forcing and step < steps - 1: # avoid last one
            #    if np.random.random() <= teacher_forcing_ratio:
            #        event = events[step].unsqueeze(0)
        
        outputs = torch.cat(outputs, 0)

        print(outputs.shape)
        print(outputA.shape)

        concatenated_vectors = torch.cat((outputA, output))
        return concatenated_vectors
        


In [57]:
init = torch.randn(batch_size, init_dim).to(device)

# Train

In [58]:
device = 'cpu'

In [59]:
model_config = {
    'init_dim': init_dim,
    'event_dim': event_dim,
    'control_dim': control_dim,
    'hidden_dim': hidden_dim,
    'gru_layers': gru_layers,
    'gru_dropout': gru_droput,
}

In [64]:
from modelF import PerformanceRNN2
from model import PerformanceRNN

In [65]:
"""
model = P2X(**model_config).to(device)
model2 = P2XSecondary(**model_config).to(device)
modelB = P2X(**model_config).to(device)
""" 
model = PerformanceRNN(**model_config).to(device)
model2 = PerformanceRNN2(**model_config).to(device)

params = list(model.parameters()) + list(model.parameters())
paramsB = list(model.parameters()) + list(model2.parameters())

optimizer = optim.Adam(paramsB, lr=learning_rate)

In [66]:
loss_function = nn.CrossEntropyLoss()

In [63]:
dataset

Dataset(root="Processed-RAW", samples=152, avglen=12133.546052631578)

In [69]:

#try:
batch_gen = dataset.batches(batch_size, window_size, stride_size)
for iteration, data in enumerate(batch_gen):
    e1 = data[0]
    c1 = data[1]
    e2 = data[2]
    c2 = data[3]

    # First Model process

    events = torch.LongTensor(e1).to(device)
    assert events.shape[0] == window_size
    assert len(events.shape) == 2
    assert events.shape[0] >= window_size - 1

    if np.random.random() < control_ratio:
        controls = torch.FloatTensor(c1).to(device)
        assert controls.shape[0] == window_size
    else:
        controls = None

    init = torch.randn(batch_size, model.init_dim).to(device)
    outputs = model.generate(init,batch_size, model.init_dim, window_size, 
                            events[:-1], controls)
    
    assert outputs.shape[:2] == events.shape[:2]
    loss = loss_function(outputs.view(-1, event_dim), events.view(-1))
    #print(outputs)

    # Second Model process

    eventsB = torch.LongTensor(e2).to(device)
    assert eventsB.shape[0] == window_size
    assert len(eventsB.shape) == 2
    assert eventsB.shape[0] >= window_size - 1

    if np.random.random() < control_ratio:
        controls = torch.FloatTensor(c2).to(device)
        assert controls.shape[0] == window_size
    else:
        controls = None

    init = torch.randn(batch_size, model2.init_dim).to(device)
    outputsB = model2.generate(init,batch_size, model2.init_dim, window_size, 
                            events=eventsB[:-1], events2=outputs[:-1], controls=controls)
    
    assert outputsB.shape[:2] == eventsB.shape[:2]
    #eventsF = torch.cat((events,eventsB))
    #outputsF = torch.cat((outputs, outputsB),2)
    #print(outputsF)

    loss = loss + loss_function(outputsB.view(-1, event_dim), eventsB.view(-1))

    model.zero_grad()
    model2.zero_grad()
    loss.backward()

    #concatenated_vectors = torch.cat((outputs, outputsB),2)
    #print(outputsF.shape)
    #print(outputsB.shape)
    #print(events.shape)

    norm = utils.compute_gradient_norm(model.parameters())
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    optimizer.step()

    print(f'iter {iteration}, loss: {loss.item()}')

    

#except:
#    print("banane")


AttributeError: 'int' object has no attribute 'shape'

In [ ]:
"""
alfa = None

batch_gen = dataset.batches(batch_size, window_size, stride_size)
for iteration, data in enumerate(batch_gen):
    e1 = data[0]
    c1 = data[1]
    e2 = data[2]
    c2 = data[3]

    events = torch.LongTensor(e1).to(device)
    assert events.shape[0] == window_size
    assert len(events.shape) == 2
    assert events.shape[0] >= window_size - 1

    print(window_size)
    print(events.shape[0])
    print(events.shape)
    print(events[0])

    if np.random.random() < control_ratio:
        controls = torch.FloatTensor(c1).to(device)
        assert controls.shape[0] == window_size
    else:
        controls = None

    init = torch.randn(batch_size, model.init_dim).to(device)
    outputs = model.generate(init,batch_size, model.init_dim, window_size, 
                            events=events[:-1], controls=controls)
    assert outputs.shape[:2] == events.shape[:2]

    print(outputs)

    events = torch.LongTensor(e2).to(device)
    assert events.shape[0] == window_size
    assert len(events.shape) == 2
    assert events.shape[0] >= window_size - 1

    print(window_size)
    print(events.shape[0])
    print(events.shape)
    print(events[0])

    if np.random.random() < control_ratio:
        controls = torch.FloatTensor(c2).to(device)
        assert controls.shape[0] == window_size
    else:
        controls = None

    outputs = model2.generate(init,batch_size, model.init_dim, window_size, 
                            events=events[:-1], controls=controls, outputA = outputs)
    
    print(outputs)
    alfa = outputs.detach().numpy().T 

    break
    """

'\nalfa = None\n\nbatch_gen = dataset.batches(batch_size, window_size, stride_size)\nfor iteration, data in enumerate(batch_gen):\n    e1 = data[0]\n    c1 = data[1]\n    e2 = data[2]\n    c2 = data[3]\n\n    events = torch.LongTensor(e1).to(device)\n    assert events.shape[0] == window_size\n    assert len(events.shape) == 2\n    assert events.shape[0] >= window_size - 1\n\n    print(window_size)\n    print(events.shape[0])\n    print(events.shape)\n    print(events[0])\n\n    if np.random.random() < control_ratio:\n        controls = torch.FloatTensor(c1).to(device)\n        assert controls.shape[0] == window_size\n    else:\n        controls = None\n\n    init = torch.randn(batch_size, model.init_dim).to(device)\n    outputs = model.generate(init,batch_size, model.init_dim, window_size, \n                            events=events[:-1], controls=controls)\n    assert outputs.shape[:2] == events.shape[:2]\n\n    print(outputs)\n\n    events = torch.LongTensor(e2).to(device)\n    asser

# Final Shot

In [ ]:
from modelF import PerformanceRNN, Guren

In [ ]:
model_config = {
    'init_dim': init_dim,
    'event_dim': event_dim,
    'control_dim': control_dim,
    'hidden_dim': hidden_dim,
    'gru_layers': gru_layers,
    'gru_dropout': gru_droput,
    'device': 'cuda'
}

In [ ]:
device = 'cuda'

In [ ]:
model = PerformanceRNN(**model_config).to(device)
model2 = PerformanceRNN(**model_config).to(device)

In [ ]:
guren_config = {
    'modelA' : model,
    'modelB' : model2
}

In [ ]:
guren = Guren(**guren_config).to(device)

In [ ]:
batch_gen = dataset.batches(batch_size, window_size, stride_size)
outputs = []
for iteration, data in enumerate(batch_gen):
    e1 = data[0]
    c1 = data[1]
    e2 = data[2]
    c2 = data[3]

    output = guren.generate(e1,e2,c1,c2,batch_size,window_size)
    outputs.append(output)
    print(iteration + " \ " +len(batch_gen))


AttributeError: 'numpy.ndarray' object has no attribute 'to'